# 5 minutes to Pandas

This notebook shows how to use [Pandas](https://pandas.pydata.org/) to clean data scraped from [realtor.com](https://www.realtor.com) with [import.io](https://import.io). The idea is then to use this data with BigML to build a real estate pricing model. Read more about the whole process on this import.io blog post: [Become a Data Scientist in an hour](http://blog.import.io/post/become-a-data-scientist-in-an-hour) (screencast inside).

The data we're working on comes from import.io. You can [browse it on Google Spreadsheets](https://docs.google.com/spreadsheets/d/12ac9fqSLdgR18pc-BUGBQeZ-xYk_BecflLHhp5A3SqE/edit?usp=sharing) and you can [download it as a CSV file](https://bml-data.s3.amazonaws.com/realtor_importio_raw.csv). When browsing the data we notice several issues:

 * some essential values are missing or some surface values are 0 => we should drop the corresponding rows
 * missing values for the number of half bathrooms should actually be 0 => we should fill in these missing values
 * there are mixed units in surface values (some small and some much larger) => we should bring all values to the same unit
 
Besides, we should run some standard procedures: dropping columns we're not interested in, removing duplicates, shuffling instances randomly (so that they are not ordered in any particular way and we can get representative subsets of the data by just splitting it).


## Reading a csv file

In [ ]:
import pandas as pd
df = pd.read_csv('https://bml-data.s3.amazonaws.com/realtor_importio_raw.csv', index_col=False, encoding='utf8') # no index column in this csv file

Let's see what the first lines of the data look like.

In [ ]:
df[:5]

## Dropping columns

In [ ]:
df = df[['address', 'bedrooms', 'full_bathrooms', 'half_bathrooms', 'type', 'size_sqft', 'lot_size', 'price']] # only keep the columns we're interested in and reorder them to have the price (target output) at the end
df[:5]

## Dropping duplicates

How many instances (rows)?

In [ ]:
len(df.index)

Let's remove duplicates...

In [ ]:
df.drop_duplicates(inplace=True)

How many instances now?

In [ ]:
len(df.index)

## Dropping bad rows

Ditch instances/rows where

 * there is no value for the number of bedrooms or for the number of bathrooms (as is the case for properties that are just land)
 * the surface is 0

In [ ]:
df.dropna(subset=['bedrooms', 'full_bathrooms', 'size_sqft'], how='any', inplace = True) # drop rows with empty values
df = df[df['size_sqft'] > 0]

How many instances now?

In [ ]:
len(df.index)

## Filling in missing values

When the number of half bathrooms is missing from the data, it actually means that there are none, i.e. there should be a zero.

In [ ]:
df['half_bathrooms'].fillna(0, inplace = True)

Let's look again at the first lines of the data.

In [ ]:
df[:5]

## Bringing all surface values to the same unit

What's the smallest livable surface?

In [ ]:
min_size_sqft = df['size_sqft'].min()
min_size_sqft

What's the smallet lot surface?

In [ ]:
df['lot_size'].min()

Some instances have a lot size value in square feet and others in acres. We need to convert everything to square feet. For this we first define a conversion function that takes a row as parameter.

In [ ]:
def convert_lot_size_surface(row):
    v = row['lot_size']
    if v < min_size_sqft: # value is too low: trigger conversion from acres to square feet
        return v * 43560 
    else:
        return v

We can then apply this function to all the rows and check what the minimum lot size value is now.

In [ ]:
df['lot_size'] = df.apply(convert_lot_size_surface, axis = 1)
df['lot_size'].min()

## Random shuffling of rows

Shuffle instances randomly so that we're sure they are not ordered in any particular way and we'll be able to create representative subsets of the data by just splitting. This is useful for Machine Learning purposes where we split into a training and a test set.

In [ ]:
import numpy as np
df = df.reindex(np.random.permutation(df.index))

Checking again the beginning of the data:

In [ ]:
df[:5]

We save the result as a new csv file.

In [ ]:
df.to_csv('realtor_importio_cleaned.csv', encoding='utf8')

## Machine Learning

We can use this data to create a Machine Learning model of real estate pricing in just 5 minutes (see other notebook in this directory).